<a href="https://colab.research.google.com/github/mkbahk/graphcore-ipu-demo/blob/main/mnist-dnn-keras-functional-cpu-input-and-output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import time
import os

In [9]:
start = time.time() # 시작 시간 저장
strategy = tf.distribute.get_strategy()

In [10]:
# The input data and labels.
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, x_test) = (x_train / 255.0, x_test / 255.0)

# Add a channels dimension.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [11]:
def create_train_dataset():
    print("==============================Processing Training DataSet==============================\n\n")
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(32, drop_remainder=True)
    train_ds = train_ds.map(lambda d, l: (tf.cast(d, tf.float32), tf.cast(l, tf.float32)))
    return train_ds.repeat()
#end of def

def create_test_dataset():
    print("==============================Processing Test  DataSet==============================\n\n")
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(32, drop_remainder=True)
    test_ds = test_ds.map(lambda d, l: (tf.cast(d, tf.float32), tf.cast(l, tf.float32)))
    return test_ds.repeat()
#end of def

In [34]:
def create_model(width=28, height=28, depth=3):
    inputs = tf.keras.Input(shape=(width, height, depth))

    x = keras.layers.Dense(128, activation='relu')(inputs)
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    
    # Defined the model.
    model = tf.keras.Model(inputs, outputs, name="dnn")
    return model
#end of def

In [35]:
def main():
      # Get the training dataset.
      print("==============================Getting Training DataSet==============================\n\n")
      ds1 = create_train_dataset()
      print("==============================Getting Test DataSet==============================\n\n")
      ds2 = create_test_dataset()

      with strategy.scope():   
        # Create an instance of the model.
        print("==============================Building Model==============================\n\n")
        model = create_model(width=28, height=28, depth=3)
        
        model.summary()

        print("==============================Building Compile==============================\n\n")      
        model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                  optimizer = tf.keras.optimizers.Adam(), 
                  steps_per_execution = 50, 
                  metrics=['sparse_categorical_accuracy'])
        
        print("==============================Model Training ==============================\n\n")
        model.fit(ds1, steps_per_epoch=2000, epochs=50)

        print("\n\n==============================Checking the result==============================\n\n")
        loss, accuracy = model.evaluate(ds2, steps=1000)
        print("Validation loss: {}".format(loss))
        print("Validation accuracy: {}%".format(100.0 * accuracy))
        print("\n\n==============================Job Done==============================")
        #end of with
#end of def

In [36]:
if __name__ == '__main__':
    main()
#end of if

==============================Getting Training DataSet==============================


==============================Processing Training DataSet==============================


==============================Getting Test DataSet==============================


==============================Processing Test  DataSet==============================


==============================Building Model==============================


Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 3)]       0         
_________________________________________________________________
dense_32 (Dense)             (None, 28, 28, 128)       512       
_________________________________________________________________
dense_33 (Dense)             (None, 28, 28, 256)       33024     
_________________________________________________________________
dense_34 (Dense)             (None,

ValueError: ignored

In [31]:
print("실행시간 :", time.time() - start,"(초)")  # 현재시각 - 시작시간 = 실행 시간

실행시간 : 1124.29882645607 (초)
